<a href="https://colab.research.google.com/github/EnterTheBeginning/Machine-Learning/blob/master/HW2_WinnerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from keras.utils import np_utils
from keras.models import Sequential
from scipy.misc import toimage
from keras.layers import Dense, Activation, Flatten,Conv2D,MaxPooling2D,Dropout, BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

Using TensorFlow backend.


In [0]:
from keras.datasets import cifar10 
(X_train_original, y_train_original), (X_test_original, y_test_original) = cifar10.load_data()


In [0]:
#OneHotEncoding
y_train = np_utils.to_categorical(y_train_original,10)
y_test = np_utils.to_categorical(y_test_original,10)


In [0]:
#Normalize
x_train = X_train_original.astype('float32')
x_test = X_test_original.astype('float32')


In [0]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
weight_decay = 1e-4


In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# model.summary()
 
#data augmentation

 
#training
 


In [0]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [0]:
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test))
#save to disk
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Instructions for updating:
Use tf.cast instead.
Epoch 1/125
781/781 [==============================] - 40s 51ms/step - loss: 1.8877 - acc: 0.4332 - val_loss: 1.3390 - val_acc: 0.5789
Epoch 2/125
781/781 [==============================] - 35s 45ms/step - loss: 1.2795 - acc: 0.5900 - val_loss: 1.0017 - val_acc: 0.6875
Epoch 3/125
781/781 [==============================] - 36s 46ms/step - loss: 1.0836 - acc: 0.6538 - val_loss: 1.0098 - val_acc: 0.6952
Epoch 4/125
781/781 [==============================] - 34s 44ms/step - loss: 0.9818 - acc: 0.6894 - val_loss: 0.8940 - val_acc: 0.7305
Epoch 5/125
781/781 [==============================] - 36s 46ms/step - loss: 0.9170 - acc: 0.7131 - val_loss: 0.8398 - val_acc: 0.7504
Epoch 6/125
781/781 [==============================] - 35s 45ms/step - loss: 0.8635 - acc: 0.7365 - val_loss: 0.8024 - val_acc: 0.7672
Epoch 7/125
781/781 [==============================] - 35s 44ms/step - loss: 0.8302 - acc: 0.7484 - val_loss: 0.7539 - val_acc: 0.7761
Epoch 8

In [0]:
acc = model.history['acc']
val_acc = model.history['val_acc']

loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(1, len(acc) + 1)

# training and validation accuracy

plt.plot(epochs, acc, 'bo', label='training acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.title('training and validation accuracy')
plt.legend()

plt.figure()

# training and validation loss

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.title('training and validation loss')
plt.legend()

plt.show()

##K-fold